In [11]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI

api_key = os.getenv("GOOGLE_API_KEY")
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", api_key=api_key)

In [4]:
llm.invoke("What are the clubs that students can join at INPT Rabat")

AIMessage(content='Unfortunately, a definitive and up-to-date list of student clubs at INPT Rabat (Institut National des Postes et Télécommunications de Rabat) isn\'t readily available online through easily accessible public sources.  Information about student clubs at universities in Morocco is often not centrally compiled and publicized in English.\n\nTo find this information, I recommend trying these approaches:\n\n* **Check the INPT Rabat Website:** Look for a student portal, student life section, or a dedicated page for extracurricular activities.  Sometimes universities will list clubs affiliated with them.\n\n* **Contact INPT Rabat Directly:** The best way to get accurate information would be to contact the student affairs office or a student organization at INPT Rabat.  Their website should have contact details.\n\n* **Social Media:** Search for "INPT Rabat clubs" or similar terms on Facebook, Instagram, or other social media platforms. Students often advertise their clubs ther

In [15]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain_huggingface import HuggingFaceEmbeddings


loader = TextLoader(r'C:\Users\hp\Downloads\RAG-Application\Data\inpt.txt')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=4)
docs = text_splitter.split_documents(documents)

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [17]:
import os
from pinecone import Pinecone, ServerlessSpec
# Initialize Pinecone
pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

index_name = "rag-app"

# Check if the index exists, and create it if it doesn't
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
    print(f"Index '{index_name}' created.")
else:
    print(f"Index '{index_name}' already exists.")

# Connect to the index
index = pc.Index(index_name)

Index 'rag-app' already exists.


In [18]:
# Create vector store
from langchain.docstore.document import Document
from langchain_pinecone import PineconeVectorStore
vectorstore = PineconeVectorStore.from_documents(
    documents=docs,
    embedding=embeddings,
    index_name=index_name
)


In [19]:
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain_core.runnables import RunnableMap

template = """
You are an assistant specializing in the Institut National des Postes et Télécommunications (INPT).
Your role is to provide information and answer questions related only to INPT, including its programs, research, and organizational details.
If a question is outside your area of expertise, politely inform the user that you can only assist with questions related to INPT.

Context: {context}
Question: {question}
Answer:
"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=template
)

# RAG chain definition
rag_chain = (
    RunnableMap({
        "context": vectorstore.as_retriever(), 
        "question": RunnablePassthrough()
    })
    | prompt
    | llm  
    | StrOutputParser()
)


In [22]:
question = "What are the clubs that students can join at INPT Rabat? Give me two clubs with a brief explanation of each one's activities."
response = rag_chain.invoke(question)
print(response)


Students at INPT Rabat can join several clubs.  Here are two examples:

* **CAS (Club des Affaires Sociales):** This club focuses on impactful social initiatives and community outreach.

* **CIT (Club d’Innovation et de Technologie):** This club hosts workshops in data science, software development, cybersecurity, and DevOps.  They also run nationwide competitions like JNJD and IDEH for developers across Morocco.
